In [22]:
!pip install --quiet tensorflow_text==2.11.*

In [23]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [24]:
df=pd.read_csv("https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/47_BERT_text_classification/spam.csv")

In [25]:
BERT_MODEL = "https://tfhub.dev/google/experts/bert/wiki_books/2" # @param {type: "string"} ["https://tfhub.dev/google/experts/bert/wiki_books/2", "https://tfhub.dev/google/experts/bert/wiki_books/mnli/2", "https://tfhub.dev/google/experts/bert/wiki_books/qnli/2", "https://tfhub.dev/google/experts/bert/wiki_books/qqp/2", "https://tfhub.dev/google/experts/bert/wiki_books/squad2/2", "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2",  "https://tfhub.dev/google/experts/bert/pubmed/2", "https://tfhub.dev/google/experts/bert/pubmed/squad2/2"]
# Preprocessing must match the model, but all the above use the same.
PREPROCESS_MODEL = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [26]:
preproess_model=hub.KerasLayer(PREPROCESS_MODEL)
bert_model=hub.KerasLayer(BERT_MODEL)

In [27]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
df["Category"]=df["Category"].apply(lambda x:1 if x=="spam" else 0)

In [29]:
df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df[["Message"]],df[["Category"]],test_size=0.2,stratify=df["Category"])

In [34]:
text_input=keras.layers.Input(shape=(),dtype=tf.string,name="input")
preproess_text=preproess_model(text_input)
output=bert_model(preproess_text)
l=tf.keras.layers.Dropout(0.1,name="Dropout")(output["pooled_output"])
l=tf.keras.layers.Dense(1,activation="sigmoid",name="output")(l)
model=tf.keras.Model(inputs=[text_input],outputs=[l])

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 keras_layer_14 (KerasLayer)    {'input_type_ids':   0           ['input[0][0]']                  
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [36]:
model.compile(optimizer="adam",loss="BinaryCrossentropy",metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Precision()])


In [37]:
model.fit(X_train,y_train)

  3/140 [..............................] - ETA: 24:27 - loss: 0.4531 - binary_accuracy: 0.8438 - precision: 0.2500

KeyboardInterrupt: 